In [1]:
%pip install requests beautifulsoup4 newspaper3k datetime pandas


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/7.4 MB ? eta -:--:--
     - -------------------------------------- 0.3/7.4 MB ? eta -:--:--
     -- ------------------------------------- 0.5/7.4 MB 2.1 MB/s eta 0:00:04
     ----- ---------------------------------- 1.0/7.4 MB 1.7 MB/s eta 0:00:04
     ------- -------------------------------- 1.3/7.4 MB 1.6 MB/s eta 0:00:04
     -------- ------------------------------- 1.6/7.4 MB 1.6 MB/s eta 0:00:04
     ----------- ---------------------------- 2.1/7.4 MB 1.7 MB/s eta 0:00:04
     ------------ --------------------------- 2.4/7.4 MB 1.6 MB/s eta 0:00:04
     -------------- ------------------------- 2.6/7.4 MB 1.6 MB/s eta 0:00:04
     --------------- ------------------------ 2.9/7.4 MB 1.6 MB/s eta 0:00:03
     ------------------ --------------------- 3.4/7.4 MB 1.6 MB/s eta 0:00:03
     ------------------- -------------------- 3.7/7.4

In [2]:
%pip install huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [5]:
!huggingface-cli login

^C


In [4]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert')
finbert = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert')
nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

def FinBERT_sentiment_score(headings):
    """
    Compute sentiment score using pretrained FinBERT on -1 to 1 scale for a batch of headings.
    -1 being negative and 1 being positive
    """
    results = nlp(headings)
    scores = []
    for result in results:
        if result['label'] == "positive":
            scores.append(result['score'])
        elif result['label'] == "neutral":
            scores.append(0)
        else:
            scores.append((0 - result['score']))
    return scores

news_df = pd.read_csv("finaldata.csv")
batch_size = 100  # You can adjust this based on your memory limitations
finbert_sentiment = []
for i in range(0, len(news_df), batch_size):
    batch_headings = []
    for j in range(i, min(i + batch_size, len(news_df))):
        news_list = news_df.iloc[j, 1:].tolist()
        news_list = [i for i in news_list if i != '0']
        heading = ' '.join(news_list)
        batch_headings.append(heading)
    scores = FinBERT_sentiment_score(batch_headings)
    finbert_sentiment.extend(scores)

news_df['FinBERT score'] = finbert_sentiment

news_df.to_csv("sentimentmerged1.csv")

c:\Users\Asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: [WinError 126] The specified module could not be found. Error loading "c:\Users\Asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.